In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

In [8]:
import numpy as np
import pandas as pd
import os # 모델 저장할 폴더가 있는지 여부와 폴더 생성

from tensorflow.keras.datasets import mnist # mnist 훈련셋과 테스트셋
from tensorflow.keras.models import Sequential # 모델 객체 생성
from tensorflow.keras.utils import to_categorical # 분류분석시 y데이터 원핫인코딩
from tensorflow.keras.layers import Input, Dense, Dropout # 모델 layer 쌓을 때 필요. 과적합시 필요
from tensorflow.keras.metrics import Recall, Precision # accuracy 외에 정밀도, 재현율 설정
from tensorflow.keras.models import save_model, load_model # 모델 저장, 로드
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint # 로그출력 간소화, epoch다 돌아가기 전에 괜찮으면 끝내는것

from sklearn.preprocessing import LabelEncoder # 라벨인코더
from sklearn.metrics import confusion_matrix # 혼동행렬, 교차표
from sklearn.model_selection import train_test_split # 데이터 분리
from sklearn import datasets # iris 데이터(X,y) 가져오기

import matplotlib.pyplot as plt
import seaborn as sns # iris 데이터(데이터프레임) 가져오기

# 1. 기본적인 DNN

In [12]:
# 1. 데이터 생성 및 전처리
# datasets.load_iris : 라벨인코딩까지 되어있음

iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target

train_X, test_X, train_y, test_y = train_test_split(iris_X, iris_y,
                                                    test_size=0.2,
                                                    stratify=iris_y)
train_X.shape, train_y.shape, test_X.shape, test_y.shape

((120, 4), (120,), (30, 4), (30,))

In [16]:
# 2. 모델 구성

# 기존방식
# model.add(Input(4))
# model.add(Dense(units=64, activation='relu'))
# model.add(Dense(units=125, activation='relu'))
# model.add(Dense(units=50, activation='relu'))
# model.add(Dense(units=30, activation='relu'))
# model.add(Dense(units=3, activation='softmax'))
# model.summary()

# 요즘 추세
model = Sequential([Input(4), 
                    Dense(units=64, activation='relu'),
                    Dense(units=125, activation='relu'),
                    Dense(units=50, activation='relu'),
                    Dense(units=30, activation='relu'),
                    Dense(units=3, activation='softmax')])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 64)                320       
                                                                 
 dense_11 (Dense)            (None, 125)               8125      
                                                                 
 dense_12 (Dense)            (None, 50)                6300      
                                                                 
 dense_13 (Dense)            (None, 30)                1530      
                                                                 
 dense_14 (Dense)            (None, 3)                 93        
                                                                 
Total params: 16,368
Trainable params: 16,368
Non-trainable params: 0
_________________________________________________________________


In [17]:
# 3. 학습과정 설정

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 4. 모델 학습
earlystopping = EarlyStopping(patience=40)
hist = model.fit(train_X, train_y,
                 epochs=200,
                 validation_split=0.2,
                 verbose=1,
                 callbacks=[earlystopping])

Epoch 1/200
3/3 [==============================] - 0s 62ms/step - loss: 1.0194 - accuracy: 0.5312 - val_loss: 0.9399 - val_accuracy: 0.9583
Epoch 2/200
3/3 [==============================] - 0s 14ms/step - loss: 0.9264 - accuracy: 0.6771 - val_loss: 0.8798 - val_accuracy: 0.8333
Epoch 3/200
3/3 [==============================] - 0s 12ms/step - loss: 0.8428 - accuracy: 0.7604 - val_loss: 0.8185 - val_accuracy: 0.5417
Epoch 4/200
3/3 [==============================] - 0s 14ms/step - loss: 0.7695 - accuracy: 0.7083 - val_loss: 0.7221 - val_accuracy: 0.7917
Epoch 5/200
3/3 [==============================] - 0s 13ms/step - loss: 0.6961 - accuracy: 0.9167 - val_loss: 0.6261 - val_accuracy: 1.0000
Epoch 6/200
3/3 [==============================] - 0s 13ms/step - loss: 0.6361 - accuracy: 0.8438 - val_loss: 0.5464 - val_accuracy: 0.9167
Epoch 7/200
3/3 [==============================] - 0s 13ms/step - loss: 0.5801 - accuracy: 0.8646 - val_loss: 0.5049 - val_accuracy: 1.0000
Epoch 8/200
3/3 [===

3/3 [==============================] - 0s 12ms/step - loss: 0.0821 - accuracy: 0.9688 - val_loss: 0.1110 - val_accuracy: 0.9583
Epoch 60/200
3/3 [==============================] - 0s 15ms/step - loss: 0.0631 - accuracy: 0.9896 - val_loss: 0.0491 - val_accuracy: 1.0000
Epoch 61/200
3/3 [==============================] - 0s 13ms/step - loss: 0.0693 - accuracy: 0.9792 - val_loss: 0.1461 - val_accuracy: 0.8750
Epoch 62/200
3/3 [==============================] - 0s 12ms/step - loss: 0.0659 - accuracy: 0.9688 - val_loss: 0.1124 - val_accuracy: 0.9583
Epoch 63/200
3/3 [==============================] - 0s 13ms/step - loss: 0.0777 - accuracy: 0.9688 - val_loss: 0.0610 - val_accuracy: 1.0000
Epoch 64/200
3/3 [==============================] - 0s 14ms/step - loss: 0.0608 - accuracy: 0.9792 - val_loss: 0.1695 - val_accuracy: 0.8750
Epoch 65/200
3/3 [==============================] - 0s 12ms/step - loss: 0.0705 - accuracy: 0.9792 - val_loss: 0.0943 - val_accuracy: 0.9583
Epoch 66/200
3/3 [========

3/3 [==============================] - 0s 13ms/step - loss: 0.0550 - accuracy: 0.9792 - val_loss: 0.1666 - val_accuracy: 0.8750
Epoch 118/200
3/3 [==============================] - 0s 14ms/step - loss: 0.0537 - accuracy: 0.9688 - val_loss: 0.0885 - val_accuracy: 0.9583
Epoch 119/200
3/3 [==============================] - 0s 13ms/step - loss: 0.0542 - accuracy: 0.9792 - val_loss: 0.0403 - val_accuracy: 1.0000
Epoch 120/200
3/3 [==============================] - 0s 12ms/step - loss: 0.0603 - accuracy: 0.9792 - val_loss: 0.1376 - val_accuracy: 0.8750
Epoch 121/200
3/3 [==============================] - 0s 12ms/step - loss: 0.0562 - accuracy: 0.9688 - val_loss: 0.1463 - val_accuracy: 0.8750
Epoch 122/200
3/3 [==============================] - 0s 13ms/step - loss: 0.0503 - accuracy: 0.9688 - val_loss: 0.0577 - val_accuracy: 0.9583
Epoch 123/200
3/3 [==============================] - 0s 14ms/step - loss: 0.0605 - accuracy: 0.9792 - val_loss: 0.0586 - val_accuracy: 0.9583
Epoch 124/200
3/3 [=

# 2. sklearn 이용

- 원핫인코딩을 하지 않고 라벨인코딩까지만 해야 작동

In [18]:
from sklearn.neural_network import MLPClassifier

In [19]:
# 1. 데이터

train_X.shape, train_y.shape, test_X.shape, test_y.shape

((120, 4), (120,), (30, 4), (30,))

In [20]:
model = MLPClassifier(hidden_layer_sizes=(64, 128, 50, 30), # hidden layer의 units수
                      activation='relu',       # 마지막 softmax는 알아서 입힘
                      solver='adam',           # compile도 알아서 해줌. (= optimizer)
                      batch_size=40,
                      max_iter=1000,           # 학습 최대 횟수. (= epoch)
                      early_stopping=True,     # 조기 종료 활성화
                      n_iter_no_change=20,     # (= patience)
                      validation_fraction=0.1, # 검증셋 비율 (= validation_split)
                      warm_start=False         # True일 경우 이전학습에 이어서 학습
                      )

In [21]:
model.fit(train_X, train_y)

MLPClassifier(batch_size=40, early_stopping=True,
              hidden_layer_sizes=(64, 128, 50, 30), max_iter=1000,
              n_iter_no_change=20)

In [23]:
# (= model.evaluate)

model.score(test_X, test_y)*100

96.66666666666667

In [29]:
iris_X[145]

array([6.7, 3. , 5.2, 2.3])

In [30]:
# 모델 예측

input_data = [6.7, 3. , 5.2, 2.3]
model.predict([input_data])

array([2])

In [31]:
# 교차표

# 실제값
test_y 

# 예측값
hat_y = model.predict(test_X)

pd.crosstab(test_y, hat_y, rownames=['real'], colnames=['predict'])

predict,0,1,2
real,,,
0,10,0,0
1,0,10,0
2,0,1,9


# 3. 클래스를 생성하여 모델 생성함수 사용

In [32]:
class DNNClassifier:
    @staticmethod
    def build(input_dim=4, activation='relu'):
        
        # 모델 구성
        model = Sequential([Input(input_dim),
                            Dense(units=50, activation=activation),
                            Dense(units=30, activation=activation),
                            Dense(units=3, activation='softmax')])
        # 학습 설정
        model.compile(loss='sparse_categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
        return model

In [35]:
# 1. 데이터
train_X.shape, train_y.shape, test_X.shape, test_y.shape

# 2. 모델
model = DNNClassifier.build(input_dim=4, activation='elu') #input_dim=4, activation='relu'일경우 아무것도 안써도 가능

# 3. 모델 학습
hist = model.fit(train_X, train_y, epochs=100,
                 validation_split = 0.1)

Epoch 1/100
4/4 [==============================] - 0s 41ms/step - loss: 1.0182 - accuracy: 0.3333 - val_loss: 0.8888 - val_accuracy: 0.6667
Epoch 2/100
4/4 [==============================] - 0s 9ms/step - loss: 0.8920 - accuracy: 0.6296 - val_loss: 0.8045 - val_accuracy: 0.6667
Epoch 3/100
4/4 [==============================] - 0s 10ms/step - loss: 0.8357 - accuracy: 0.6667 - val_loss: 0.7531 - val_accuracy: 0.6667
Epoch 4/100
4/4 [==============================] - 0s 10ms/step - loss: 0.7528 - accuracy: 0.7500 - val_loss: 0.7254 - val_accuracy: 1.0000
Epoch 5/100
4/4 [==============================] - 0s 9ms/step - loss: 0.7001 - accuracy: 0.9815 - val_loss: 0.6769 - val_accuracy: 1.0000
Epoch 6/100
4/4 [==============================] - 0s 9ms/step - loss: 0.6507 - accuracy: 0.9630 - val_loss: 0.6169 - val_accuracy: 1.0000
Epoch 7/100
4/4 [==============================] - 0s 9ms/step - loss: 0.6010 - accuracy: 0.9630 - val_loss: 0.5618 - val_accuracy: 0.9167
Epoch 8/100
4/4 [=======

4/4 [==============================] - 0s 9ms/step - loss: 0.1000 - accuracy: 0.9815 - val_loss: 0.0790 - val_accuracy: 1.0000
Epoch 60/100
4/4 [==============================] - 0s 9ms/step - loss: 0.1003 - accuracy: 0.9815 - val_loss: 0.0809 - val_accuracy: 1.0000
Epoch 61/100
4/4 [==============================] - 0s 9ms/step - loss: 0.0976 - accuracy: 0.9815 - val_loss: 0.0786 - val_accuracy: 1.0000
Epoch 62/100
4/4 [==============================] - 0s 8ms/step - loss: 0.0955 - accuracy: 0.9815 - val_loss: 0.0814 - val_accuracy: 1.0000
Epoch 63/100
4/4 [==============================] - 0s 8ms/step - loss: 0.0944 - accuracy: 0.9815 - val_loss: 0.0806 - val_accuracy: 1.0000
Epoch 64/100
4/4 [==============================] - 0s 9ms/step - loss: 0.0980 - accuracy: 0.9722 - val_loss: 0.0980 - val_accuracy: 0.9167
Epoch 65/100
4/4 [==============================] - 0s 10ms/step - loss: 0.0927 - accuracy: 0.9907 - val_loss: 0.0799 - val_accuracy: 1.0000
Epoch 66/100
4/4 [==============

In [36]:
# 모델 평가

loss, accuracy = model.evaluate(test_X, test_y)
print(f'정확도 : {accuracy*100}%')

1/1 [==============================] - 0s 17ms/step - loss: 0.0608 - accuracy: 0.9667
정확도 : 96.66666388511658%


# 4. 함수형 API

- 병렬처리, Residual block, 

[AI설명]
- Sequential이 처리 못하는 복잡한 모델 구조를 만들 때 사용
- 다중 입력/출력: 이미지 + 텍스트 입력, 여러 예측값 출력 (예: 객체 탐지 + 클래스 분류)
- 레이어 공유: 같은 레이어를 여러 경로에서 재사용 (예: ResNet의 skip connection)
- 비선형 연결: 순차가 아닌 그래프 구조 (레이어가 분기/합병)
- 동적 모델: 런타임에 구조 변경 가능

In [38]:
# 함수형 API쓰는 이유 AI설명
pd.read_csv('data/AI-Sequential-Functional.csv')

,상황,Sequential,Functional
0,단순 직렬 (지금 코드처럼),✅ 간단,❌ 과도
1,다중 입력,❌ 불가,✅ 자유
2,레이어 공유,❌ 불가,✅ 가능
3,분기/합병,❌ 불가,✅ 가능


In [ ]:
# 기존 모델 스타일 1
model = Sequential()
model.add(Input(11)) 
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=30, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

# 기존 모델 스타일 2
model = Sequential([Input(input_dim),
                    Dense(units=50, activation='relu'),
                    Dense(units=30, activation='relu'),
                    Dense(units=3, activation='softmax')])

In [37]:
# 기존 모델 스타일 3 (함수형 API 스타일. 병렬방식X, 직렬방식)

from tensorflow.keras import Model
input_ = Input(shape=(4,))
layer1 = Dense(units=50, activation='relu')(input_)
layer2 = Dense(units=30, activation='relu')(layer1)
output = Dense(units=3, activation='softmax')(layer2)
model = Model(inputs=input_, outputs=output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 4)]               0         
                                                                 
 dense_18 (Dense)            (None, 50)                250       
                                                                 
 dense_19 (Dense)            (None, 30)                1530      
                                                                 
 dense_20 (Dense)            (None, 3)                 93        
                                                                 
Total params: 1,873
Trainable params: 1,873
Non-trainable params: 0
_________________________________________________________________


[병렬방식]

```
           ┌──> Dense(50, relu) ─┐

입력 (4) ─┼──> Dense(80, relu) ─┼──> Concatenate (160) ──> Dense(3, softmax) ──> 출력 (3)
 
           └──> Dense(30, relu) ─┘
```

In [39]:
# 병렬처리 방식 (함수형 API방식만 가능)

from tensorflow.keras.layers import concatenate, Activation
input_ = Input(shape=(4,))
dense1 = Dense(units=50, activation='relu')(input_)
dense2 = Dense(units=80, activation='relu')(input_)
dense3 = Dense(units=30, activation='relu')(input_)
x = concatenate([dense1, dense2, dense3])
output = Dense(units=3, activation='softmax')(x)
model = Model(inputs=input_, outputs=output)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 4)]          0           []                               
                                                                                                  
 dense_21 (Dense)               (None, 50)           250         ['input_6[0][0]']                
                                                                                                  
 dense_22 (Dense)               (None, 80)           400         ['input_6[0][0]']                
                                                                                                  
 dense_23 (Dense)               (None, 30)           150         ['input_6[0][0]']                
                                                                                            

[리지듀얼 블럭 AI 설명]

- Residual block은 "입력값을 그대로 다음 층으로 건너뛰어 주는" 간단한 아이디어입니다.
비유: 책 읽을 때 처음부터 끝까지 다 읽는 대신, *중요한 부분만 메모(x + F(x))*해서 기억하는 것.

- 왜 좋은가?
    - 깊은 층에서도 초기 정보 잃지 않음 (기울기 소실 방지)

    - 네트워크가 "변화만" 학습 (x는 그대로 두고 F(x)만 배움)

    - 학습이 훨씬 쉬워짐 (ResNet-152층도 문제없이 학습)​

- 결과: 층을 100개 이상 쌓아도 성능이 떨어지지 않고 오히려 좋아집니다!

In [42]:
# 리지듀얼 블럭(Residual Block) : 딥러닝에서 기울기 소실 문제로 학습이 잘 되지 않는 부분을 해결하기 위한 제안

from tensorflow.keras.layers import add
input_ = Input(shape=(4,))
dense1 = Dense(units=50, activation='relu')(input_)
dense2 = Dense(units=50, activation='relu')(dense1)
dense3 = add([dense1, dense2]) # concatenate는 더하기. add는 중첩
output = Dense(units=3, activation='softmax')(dense3)
model = Model(inputs=input_, outputs=output)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 4)]          0           []                               
                                                                                                  
 dense_28 (Dense)               (None, 50)           250         ['input_8[0][0]']                
                                                                                                  
 dense_29 (Dense)               (None, 50)           2550        ['dense_28[0][0]']               
                                                                                                  
 add_1 (Add)                    (None, 50)           0           ['dense_28[0][0]',               
                                                                  'dense_29[0][0]']         

In [43]:
# 학습과정 설정 & 학습

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_X, train_y, epochs=100, verbose=1)

Epoch 1/100
4/4 [==============================] - 0s 2ms/step - loss: 1.8566 - accuracy: 0.3333
Epoch 2/100
4/4 [==============================] - 0s 2ms/step - loss: 1.3104 - accuracy: 0.4333
Epoch 3/100
4/4 [==============================] - 0s 2ms/step - loss: 1.0963 - accuracy: 0.3333
Epoch 4/100
4/4 [==============================] - 0s 2ms/step - loss: 1.0449 - accuracy: 0.3167
Epoch 5/100
4/4 [==============================] - 0s 2ms/step - loss: 1.0233 - accuracy: 0.3333
Epoch 6/100
4/4 [==============================] - 0s 1ms/step - loss: 0.9242 - accuracy: 0.3333
Epoch 7/100
4/4 [==============================] - 0s 2ms/step - loss: 0.8273 - accuracy: 0.6583
Epoch 8/100
4/4 [==============================] - 0s 1ms/step - loss: 0.7778 - accuracy: 0.7167
Epoch 9/100
4/4 [==============================] - 0s 2ms/step - loss: 0.7336 - accuracy: 0.7167
Epoch 10/100
4/4 [==============================] - 0s 2ms/step - loss: 0.6946 - accuracy: 0.7750
Epoch 11/100
4/4 [===========

4/4 [==============================] - 0s 2ms/step - loss: 0.1398 - accuracy: 0.9667
Epoch 85/100
4/4 [==============================] - 0s 2ms/step - loss: 0.1368 - accuracy: 0.9667
Epoch 86/100
4/4 [==============================] - 0s 1ms/step - loss: 0.1383 - accuracy: 0.9750
Epoch 87/100
4/4 [==============================] - 0s 2ms/step - loss: 0.1349 - accuracy: 0.9750
Epoch 88/100
4/4 [==============================] - 0s 2ms/step - loss: 0.1348 - accuracy: 0.9750
Epoch 89/100
4/4 [==============================] - 0s 1ms/step - loss: 0.1395 - accuracy: 0.9667
Epoch 90/100
4/4 [==============================] - 0s 1ms/step - loss: 0.1315 - accuracy: 0.9750
Epoch 91/100
4/4 [==============================] - 0s 2ms/step - loss: 0.1318 - accuracy: 0.9667
Epoch 92/100
4/4 [==============================] - 0s 2ms/step - loss: 0.1278 - accuracy: 0.9750
Epoch 93/100
4/4 [==============================] - 0s 1ms/step - loss: 0.1281 - accuracy: 0.9750
Epoch 94/100
4/4 [===============

In [44]:
# 학습 평가

loss, accuracy = model.evaluate(test_X, test_y, verbose=0)
print(f'accuracy : {accuracy}')

accuracy : 0.9666666388511658
